In [1]:
import time
import configparser
import pandas as pd
import boto3
import json
import os

In [2]:
%load_ext sql

In [3]:
#Get AWS info from dwh.cfg file
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })


,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhsongify
4,DWH_DB,dwh
5,DWH_DB_USER,dwhuser
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


In [4]:
#Define Resources
ec2 = boto3.resource('ec2',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [5]:
#Check bucket for log table files and get count of files
logfiles =  s3.Bucket("udacity-dend")
i=0
for obj in logfiles.objects.filter(Prefix='log-data'):
     i+=1
print(i)

31


In [ ]:
#Check bucket for song table files and get count of files
logfiles =  s3.Bucket("udacity-dend")
j=0
for obj in logfiles.objects.filter(Prefix='song-data'):
    j=j+1
print(j)

In [5]:
if iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['RoleName']==DWH_IAM_ROLE_NAME is False:
    #Create ARN Role
    from botocore.exceptions import ClientError

    #1.1 Create the role, 
    try:
        print("1.1 Creating a new IAM Role") 
        dwhRole = iam.create_role(
            Path='/',
            RoleName=DWH_IAM_ROLE_NAME,
            Description = "Allows Redshift clusters to call AWS services on your behalf.",
            AssumeRolePolicyDocument=json.dumps(
                {'Statement': [{'Action': 'sts:AssumeRole',
                   'Effect': 'Allow',
                   'Principal': {'Service': 'redshift.amazonaws.com'}}],
                 'Version': '2012-10-17'})
        )    
    except Exception as e:
        print(e)


    print("1.2 Attaching Policy")

    iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                           PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                          )['ResponseMetadata']['HTTPStatusCode']

    print("1.3 Get the IAM role ARN")
    ARN = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
else:
    ARN = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [6]:
try:
    response=redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)
    create=False
except:
    create=True

In [7]:
#Create Redshift Cluster
if create:
    try:
        response = redshift.create_cluster(        
            #HW
            ClusterType=DWH_CLUSTER_TYPE,
            NodeType=DWH_NODE_TYPE,
            NumberOfNodes=int(DWH_NUM_NODES),

            #Identifiers & Credentials
            DBName=DWH_DB,
            ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
            MasterUsername=DWH_DB_USER,
            MasterUserPassword=DWH_DB_PASSWORD,

            #Roles (for s3 access)
            IamRoles=[ARN]  
        )
    except Exception as e:
        print(e)

In [8]:
DWH_CLUSTER_IDENTIFIER

'dwhsongify'

In [9]:
#Wait Until Cluster becomes available
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
while myClusterProps['ClusterStatus'] != 'available':
    myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
    time.sleep(30)
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhsongify
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhsongify.cqx4tkja71nz.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-4148fc39
7,NumberOfNodes,4


In [10]:
Endpoint = myClusterProps['Endpoint']['Address']
print('Endpoint::',Endpoint)
print('ARN::',ARN)

Endpoint:: dwhsongify.cqx4tkja71nz.us-west-2.redshift.amazonaws.com
ARN:: arn:aws:iam::830305220603:role/dwhRole


<b><font color="red">Add Endpoint and ARN to dwh.cfg file</font></b>

In [1]:
run create_tables.py

In [2]:
run etl.py